# MidTerm Project: NSMC(Naver Sentiment Movie Corpus) 분류

- rating_train.txt는 네이버 영화평을 0 (negative), 1(positive)로 분류해 놓은 자료이고, ratings_test.txt는 같은 영화평의 테스트용 데이터이다.
- 이 파일을 가지고 https://github.com/bentrevett/pytorch-sentiment-analysis 에 있는 pytorch sentiment analysis의 방법을 따라  nsmc 분류기를 만들어라
- training data에서 evaluation data를 나누어 사용할 수 있다.(필요시)
- training data에 나오는 영화평을 가지고 word2vec이나, FastText 등의 임베딩을 만들고 이를 테스트시 사용하라
- 제출은 주피터 노트북과 학습된 임베딩 파일 (임베딩 파일이 커서 etl에 탑재가 되지 않으면 압축을 하거나 이메일 등 다른 방법으로 제출)
- 화일 이름은 MidTermAssignment_학번_이름
- 마감: 2023년 10월 31일 화요일 오후 11시 59분 59초까지!

## 목표

- csv 파일을 읽어서 torchtext를 사용하여 데이터를 신경망에 입력가능한 꼴로 바꾸기
- 한국어 데이터 전처리를 위한 함수를 만들고 이를 torchtext에 통합하기 (**새로운 torchtext사용**)
- 이미 실습 시간에 관련 모듈 설명됨
- 직접 학습한 한국어 단어 임베딩을 torchtext에 통합하여 사용하기
- 제시된 여러 모델을 사용하여(transformers 제외) 성능을 향상 시키기
- training, evaluation 한 것을 test 데이터에 적용하여 성능을 보이기.
- predict를 사용하여 제시된 기사들의 분류 결과를 보이기
- 참고 사이트:
- [Pytorch Sentiment Analysis](https://github.com/bentrevett/pytorch-sentiment-analysis)에 관련 코드들이 있어 참조할 수 있음

## 주의사항

- 방법
1. torchtext 최신 버전 0.14.0으로 작업.
- [TorchText 최근버전 documenation](https://pytorch.org/text/0.14.0/index.html)
2. 단어 임베딩은 Gensim을 사용하여 자유롭게 구축할 수 있음(차원, 윈도우크기, 학습율 등을 자유롭게 설정)


## 전체 구현 정리
- 임베딩 방법 자세히 기술
- 데이터 전처리 자세히 기술
- rnn, lstm, cnn 등의 방법으로 기술한 것의 성능 차이 기술
- 테스트 데이터의 성능 보고
- predict한 문장들의 성능 및 분석


In [1]:
##프로그래밍 시작
## 반드시 각 모듈별로 자세히 주석을 붙이고 설명을 할 것!

# 0. 필요한 모듈 다운로드 및  import, 파일 경로(구글 드라이브 마운트)설정 

In [2]:
!pip install konlpy
!pip install mecab-python3
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 3.3 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [591 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,186 kB]
Get:10 http://archive.u

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Mecab
import torch
import torchtext
from sklearn.model_selection import train_test_split
import random
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import Vectors
from torchtext.vocab import vocab
from torchtext import data
from torchtext.data.functional import to_map_style_dataset
import numpy as np
# from torchtext.legacy import datasets
# from torchtext.legacy.data import Iterator, BucketIterator


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Vocab 만들기
-cnn_with_embedding.ipynb에 자세히 설명이 기술되어 있기에 생략한다.

In [5]:
import torchdata.datapipes as dp
import torchtext.transforms as T
import spacy
from torchtext.vocab import build_vocab_from_iterator
train_file_path = '/content/drive/MyDrive/nlp/ratings_train.txt'
test_file_path = '/content/drive/MyDrive/nlp/ratings_test.txt'

data_pipe = dp.iter.IterableWrapper([train_file_path])
data_pipe = dp.iter.FileOpener(data_pipe, mode='rb')
data_pipe = data_pipe.parse_csv(skip_lines=1, delimiter='\t', as_tuple=True)

In [6]:
for sample in data_pipe:
    print(sample)
    break

('9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0')


In [7]:
def removeAttribution(row):
    return row[1:3]
data_pipe = data_pipe.map(removeAttribution)

In [8]:
for sample in data_pipe:
    print(sample)
    break

('아 더빙.. 진짜 짜증나네요 목소리', '0')


In [9]:
import re

mecab=Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
def tokenizer(text):
    text = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", text)  # 한글과 공백만 남기고 나머지 제거
    text = re.sub('^ +', '', text)  # 시작 부분의 여러 개의 연속된 공백 제거
    tokens = mecab.morphs(text)
    tokens = [word for word in tokens if word not in stopwords]
    return tokens

In [10]:
tokenizer("아 더빙.. 진짜 짜증나네요 목소리")

['아', '더', '빙', '진짜', '짜증', '나', '네요', '목소리']

In [11]:
def yield_tokens(data_iter):
    for text, label in data_iter:
        yield tokenizer(text)

In [12]:
vocab = build_vocab_from_iterator(yield_tokens(data_pipe), min_freq=2,
                                  specials=["<unk>", "<pad>"],special_first=True,
                                  max_tokens= 25000)
vocab.set_default_index(vocab["<unk>"])

In [13]:
print(vocab.get_itos()[:100])

['<unk>', '<pad>', '영화', '다', '고', '하', '을', '보', '게', '지', '있', '없', '좋', '나', '었', '만', '는데', '너무', '봤', '적', '안', '로', '정말', '음', '것', '아', '재밌', '네요', '어', '지만', '같', '진짜', '에서', '기', '했', '네', '점', '않', '거', '았', '수', '되', '면', 'ㅋㅋ', '인', '말', '연기', '최고', '주', '내', '평점', '이런', '던', '어요', '할', '왜', '겠', '해', '스토리', 'ㅋㅋㅋ', '습니다', '듯', '아니', '드라마', '생각', '더', '그', '싶', '사람', '감동', '때', '함', '배우', '본', '까지', '보다', '뭐', '볼', '알', '만들', '내용', '감독', '라', '재미', '그냥', '시간', '재미있', '지루', '중', '잼', '재미없', '였', '년', '쓰레기', '사랑', '못', '냐', '서', '라고', '니']


In [14]:
cnt = 0
for i in data_pipe:
    print(i)
    cnt +=1
    if cnt == 10:
      break

('아 더빙.. 진짜 짜증나네요 목소리', '0')
('흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1')
('너무재밓었다그래서보는것을추천한다', '0')
('교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0')
('사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', '1')
('막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '0')
('원작의 긴장감을 제대로 살려내지못했다.', '0')
('별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네', '0')
('액션이 없는데도 재미 있는 몇안되는 영화', '1')
('왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '1')


# 2. Dataloader 만들기
-cnn_with_embedding.ipynb에 자세히 설명이 기술되어 있기에 생략한다.

In [15]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x)

In [16]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
#collate_function: process the list of samples to form a batch.
# https://androidkt.com/create-dataloader-with-collate_fn-for-variable-length-input-in-pytorch/

def custom_collate_fn(batch):
    label_list, text_list= [], []
    for (_text,_label) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)

    label_list = torch.tensor(label_list, dtype=torch.int64).unsqueeze(dim=1)
    text_list = pad_sequence(text_list, padding_value = 1)
    return text_list.to(device),label_list.to(device)

In [18]:
data_list = list(data_pipe)

# 데이터를 train set과 validation set으로 나눔
train_data, valid_data = train_test_split(data_list, test_size=0.2, random_state=42)

In [19]:
data_list[0:10]

[('아 더빙.. 진짜 짜증나네요 목소리', '0'),
 ('흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'),
 ('너무재밓었다그래서보는것을추천한다', '0'),
 ('교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'),
 ('사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', '1'),
 ('막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '0'),
 ('원작의 긴장감을 제대로 살려내지못했다.', '0'),
 ('별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네',
  '0'),
 ('액션이 없는데도 재미 있는 몇안되는 영화', '1'),
 ('왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '1')]

In [20]:
train_dataloader = DataLoader(train_data, batch_size=16,
                              shuffle=True, collate_fn=custom_collate_fn)

valid_dataloader = DataLoader(valid_data, batch_size=16,
                              shuffle=True, collate_fn=custom_collate_fn)



# 3. CNN without word2vec

CNN 모듈이 embedding을 사용한 것과 달라지는 부부은 아래 부분이 유일하다.

self.embedding = nn.Embedding(input_dim, embedding_dim)
이후 과정은 cnn_with_embedding.ipynb에 상세한 내용이 있으니까 생략한다.

In [21]:
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        #pretrained된 임베딩을 사용하지 않음
        self.embedding = nn.Embedding(input_dim, embedding_dim)

        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1,
                                              out_channels = n_filters,
                                              kernel_size = (fs, embedding_dim))
                                    for fs in filter_sizes
                                    ])

        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        #text = [batch size, sent len]

        embedded = self.embedding(text)

        #embedded = [batch size, sent len, emb dim]

        embedded = embedded.unsqueeze(1).permute(2,1,0,3)

        #embedded = [batch size, 1, sent len, emb dim]

        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]

        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]

        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]

        #pooled_n = [batch size, n_filters]

        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
        #torch.Size([450, 1, 16, 100])
        #torch.Size([450, 300])
        return self.fc(cat)

In [22]:
INPUT_DIM = len(vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)

In [24]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,620,601 trainable parameters


# 4. 생성한 CNN을 Training
이후 과정은 cnn_with_embedding.ipynb에 상세한 내용이 있으니까 생략한다.


In [25]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

In [26]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc

In [27]:
import time
a = []
b = []
def train(dataloader):
    model.train()
    epoch_loss, epoch_acc = 0, 0
    log_interval = len(dataloader)
    start_time = time.time()

    for idx, (text, label) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text)
        loss = criterion(predicted_label, label.float())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        epoch_acc += binary_accuracy(predicted_label,label).item()
        epoch_loss += loss.item()
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                            epoch_loss/len(dataloader)))
            #total_acc, total_count = 0, 0
            start_time = time.time()

    return epoch_loss / len(dataloader), epoch_acc / len(dataloader)

In [28]:
def evaluate(dataloader):
    model.eval()
    epoch_loss, epoch_acc = 0, 0
    with torch.no_grad():
        for idx, (text, label) in enumerate(dataloader):
            predicted_label = model(text)
            loss = criterion(predicted_label, label.float())
            epoch_acc += binary_accuracy(predicted_label,label).item()
            epoch_loss += loss.item()

    return epoch_loss / len(dataloader), epoch_acc / len(dataloader)

In [30]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [31]:
EPOCHS = 10

best_valid_loss = float('inf')
for epoch in range(EPOCHS):
    epoch_start_time = time.time()
    start_time = time.time()

    train_loss, train_acc = train(train_dataloader)
    valid_loss, valid_acc = evaluate(valid_dataloader)
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'cnn_wo_w2v.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 2m 4s
	Train Loss: 0.724 | Train Acc: 50.39%
	 Val. Loss: 0.692 |  Val. Acc: 52.22%
Epoch: 02 | Epoch Time: 2m 5s
	Train Loss: 0.719 | Train Acc: 50.95%
	 Val. Loss: 0.689 |  Val. Acc: 53.46%
Epoch: 03 | Epoch Time: 2m 6s
	Train Loss: 0.714 | Train Acc: 51.52%
	 Val. Loss: 0.687 |  Val. Acc: 55.09%
Epoch: 04 | Epoch Time: 2m 6s
	Train Loss: 0.708 | Train Acc: 52.15%
	 Val. Loss: 0.684 |  Val. Acc: 56.38%
Epoch: 05 | Epoch Time: 2m 6s
	Train Loss: 0.704 | Train Acc: 52.66%
	 Val. Loss: 0.682 |  Val. Acc: 57.42%
Epoch: 06 | Epoch Time: 2m 7s
	Train Loss: 0.701 | Train Acc: 52.94%
	 Val. Loss: 0.680 |  Val. Acc: 58.44%
Epoch: 07 | Epoch Time: 2m 7s
	Train Loss: 0.697 | Train Acc: 53.44%
	 Val. Loss: 0.678 |  Val. Acc: 59.35%
Epoch: 08 | Epoch Time: 2m 5s
	Train Loss: 0.694 | Train Acc: 54.17%
	 Val. Loss: 0.677 |  Val. Acc: 60.02%
Epoch: 09 | Epoch Time: 2m 6s
	Train Loss: 0.693 | Train Acc: 54.27%
	 Val. Loss: 0.675 |  Val. Acc: 60.66%
Epoch: 10 | Epoch Time: 2m 5

# 5. 테스트 데이터 evaluate 
이후 과정은 cnn_with_embedding.ipynb에 상세한 내용이 있으니까 생략한다.


In [32]:
test_file_path = '/content/drive/MyDrive/nlp/ratings_test.txt'

test_data_pipe = dp.iter.IterableWrapper([test_file_path])
test_data_pipe = dp.iter.FileOpener(test_data_pipe, mode='rb')
test_data_pipe = test_data_pipe.parse_csv(skip_lines=1, delimiter='\t', as_tuple=True)
test_data_pipe = test_data_pipe.map(removeAttribution)

test_dataloader = DataLoader(list(test_data_pipe), batch_size=16,
                              shuffle=True, collate_fn=custom_collate_fn)
model.eval()
test_loss, test_acc = evaluate(test_dataloader)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.674 | Test Acc: 61.50%


# 6. Inference
이후 과정은 cnn_with_embedding.ipynb에 상세한 내용이 있으니까 생략한다.


- 다음 문장들을 모델에 넣었을 때 그 결과를 도출하는 inference 작성
- 0(부정)/1(긍정)
- 아래 문장의 정답은 1/1/0/0/1/1/1/0/0/0


In [33]:
model.load_state_dict(torch.load('/content/cnn_wo_w2v.pt'))

<All keys matched successfully>

In [36]:
def predict_nsmc(model, sentence, min_len=5):
  threshold=0.5
  hey=[(sentence,'0')]
  hey_dataloader = DataLoader(hey, batch_size=1,
                                shuffle=True, collate_fn=custom_collate_fn)
  model.eval()
  for idx, (text, label) in enumerate(hey_dataloader):
        prediction = torch.sigmoid(model(text))
        print(prediction.item()," : ", sentence)
        prediction = 1 if prediction.item() >= threshold else 0

  return prediction

In [37]:
result=[]
sentence="잔잔한 감동... 오래 기억에 남아요... 우리 시대의 평범한 가족 모습이 솔직하게 담겨 있네요. 그리고 한 소녀의 성장기도 아름답게 표현했습니다."
result.append(predict_nsmc(model, sentence))
sentence="한 사람의 삶을 한 화면에서 완전히 이해할 수 없다는 것을 더욱 명확하게 깨닫게 해주는 작품입니다. 밴 애플렉과 션 윌의 연기는 볼만하고, 복 받은 캐릭터들이 감동을 전해줍니다."
result.append(predict_nsmc(model, sentence))
sentence="티비 드라마보다 못한 영화. 그 당시에는 티비 드라마가 한국 영화보다 더 흥미로웠다."
result.append(predict_nsmc(model, sentence))
sentence="이 작품은 클래식을 위한 드라마나 청춘 성장 드라마와는 다른 주제를 다루고 있는데, 그 주제가 명확하지 않아 혼란스럽습니다. 현실을 고발하려는 의도는 알겠지만, 이 작품은 다소 심각한 드라마 중 하나입니다. 부채도사는 어디로..ㅠㅠ"
result.append(predict_nsmc(model, sentence))
sentence="이 작품을 통해 그들의 쾌락적인 삶을 엿볼 수 있고, 우리는 다른 형태의 쾌락을 느낍니다. 그들이 추잡하게 보일 수 있지만, 결국 우리도 돈을 추구하며 부유한 삶을 상상하는 것이 현실입니다. 이 영화는 감독의 메시지를 느끼게 합니다."
result.append(predict_nsmc(model, sentence))
sentence="미식축구를 다시 생각하게 만든 감동적인 영화입니다. 개인적으로 감명 깊었습니다."
result.append(predict_nsmc(model, sentence))
sentence="의외로 흥미로운 영화로, 지루하지 않은 사극 멜로였습니다. 캐릭터들이 매력적으로 표현되었어요."
result.append(predict_nsmc(model, sentence))
sentence="캐릭터들 중에서 할아버지를 제외하고는 짜증을 유발하는 매력적인 캐릭터가 없다는 점이 아쉽습니다."
result.append(predict_nsmc(model, sentence))
sentence="엔딩 임팩트가 부족합니다. 더 흥미로웠다면 좋았을 것 같아요."
result.append(predict_nsmc(model, sentence))
sentence="주연배우들이 지루하고, 압축된 스토리가 조금 부족한 것 같습니다. 으윽."
result.append(predict_nsmc(model, sentence))

print(result)

0.5117886066436768  :  잔잔한 감동... 오래 기억에 남아요... 우리 시대의 평범한 가족 모습이 솔직하게 담겨 있네요. 그리고 한 소녀의 성장기도 아름답게 표현했습니다.
0.4857374429702759  :  한 사람의 삶을 한 화면에서 완전히 이해할 수 없다는 것을 더욱 명확하게 깨닫게 해주는 작품입니다. 밴 애플렉과 션 윌의 연기는 볼만하고, 복 받은 캐릭터들이 감동을 전해줍니다.
0.49689334630966187  :  티비 드라마보다 못한 영화. 그 당시에는 티비 드라마가 한국 영화보다 더 흥미로웠다.
0.4654659628868103  :  이 작품은 클래식을 위한 드라마나 청춘 성장 드라마와는 다른 주제를 다루고 있는데, 그 주제가 명확하지 않아 혼란스럽습니다. 현실을 고발하려는 의도는 알겠지만, 이 작품은 다소 심각한 드라마 중 하나입니다. 부채도사는 어디로..ㅠㅠ
0.5079962611198425  :  이 작품을 통해 그들의 쾌락적인 삶을 엿볼 수 있고, 우리는 다른 형태의 쾌락을 느낍니다. 그들이 추잡하게 보일 수 있지만, 결국 우리도 돈을 추구하며 부유한 삶을 상상하는 것이 현실입니다. 이 영화는 감독의 메시지를 느끼게 합니다.
0.5342099070549011  :  미식축구를 다시 생각하게 만든 감동적인 영화입니다. 개인적으로 감명 깊었습니다.
0.4998101592063904  :  의외로 흥미로운 영화로, 지루하지 않은 사극 멜로였습니다. 캐릭터들이 매력적으로 표현되었어요.
0.3977589011192322  :  캐릭터들 중에서 할아버지를 제외하고는 짜증을 유발하는 매력적인 캐릭터가 없다는 점이 아쉽습니다.
0.541424036026001  :  엔딩 임팩트가 부족합니다. 더 흥미로웠다면 좋았을 것 같아요.
0.4865374267101288  :  주연배우들이 지루하고, 압축된 스토리가 조금 부족한 것 같습니다. 으윽.
[1, 0, 0, 0, 1, 1, 0, 0, 1, 0]
